# PydanticOutputParser

`PydanticOutputParser`는 언어 모델의 출력을 **구조화된 정보**로 변환하는 데 도움을 주는 클래스입니다. 이 클래스는 단순 텍스트 응답 대신 **명확하고 체계적인 형태로 필요한 정보를 제공**할 수 있습니다.

이 클래스를 활용하면 언어 모델의 출력을 특정 데이터 모델에 맞게 변환하여 정보를 더 쉽게 처리하고 활용할 수 있습니다.

## 주요 메서드

`PydanticOutputParser` (대부분의 OutputParser에 해당)에는 주로 **두 가지 핵심 메서드**가 구현되어야 합니다.

- **`get_format_instructions()`**: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공합니다. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 이 지침은 언어 모델이 출력을 구조화하고 특정 데이터 모델에 맞게 변환하는 데 매우 중요합니다.
- **`parse()`**: 언어 모델의 출력(문자열로 가정)을 받아 이를 특정 구조로 분석하고 변환합니다. Pydantic과 같은 도구를 사용하여 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다.

## 참고 자료

- [Pydantic 공식 도큐먼트](https://docs.pydantic.dev/latest/)

> 

In [1]:
from dotenv import load_dotenv
from langchain.chains.summarize.refine_prompts import prompt_template

load_dotenv()

True

In [3]:
techtalk_message = """작성자: 민정
:spiral_calendar_pad:  2:03 PM
:loudspeaker: 2025년 세 번째 테크톡에 여러분을 초대합니다! :tada:
:tulip: 봄바람 살랑이는 3월, 새로운 시작을 테크톡과 함께해보아요!
       이번에는 A팀 & D팀 @정호 님의 AI 에 관련된 흥미로운 발표가 준비되어 있습니다!!!
:microphone: 발표 주제  "AI 패러다임 변화와 우리의 미래" : 이정호
:date: 일정: 2025년 3월 7일 (금) 오후 2시 30분 ~ 4시 00분
:round_pushpin: 장소: 오프라인 - 세미나실 / 온라인 - ZOOM  (사내에 계시다면 세미나실로 참석 부탁드립니다! :two_hearts:)
:cherry_blossom: 참석 포인트!
      :white_check_mark: Tech-Talk 책갈피에서 지난 발표 자료와 영상도 확인 가능! (Slack #all-notices 채널)
      :white_check_mark: 전 직원 대상 행사 :confetti_ball: 참석만 해도 새로운 인사이트 + 동료와 교류 기회 :gift:
:love_letter: 2025년 세 번째 테크톡, 놓치면 아쉽겠죠? 부담 없이 참여하고 유익한 인사이트 가져가세요! 다 함께 만드는 멋진 시작, 기대하겠습니다! :muscle::sparkles:
"""

## with_structured_output()

`.with_structured_output(Pydantic)`을 사용하여 출력 파서를 추가하면, 출력을 Pydantic 객체로 변환할 수 있습니다.

In [41]:
class TechTalk(BaseModel):
    speaker: str = Field(description="발표자")
    subject: str = Field(description="발표 주제")
    date: str = Field(description="발표 날짜와 시작 시간 (yyyy-mm-dd hh:mm:ss)")

In [42]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant.
Please answer the following questions in KOREAN.

QUESTION:
{question}

PRESENTATION INFORMATION:
{techtalk_message}
"""
)

In [43]:
llm = ChatOpenAI(
    temperature=0, model_name="gpt-4o-mini"
)
llm_with_structured = llm.with_structured_output(TechTalk)

In [44]:
chain = prompt | llm_with_structured

In [45]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = chain.invoke({
    "techtalk_message": techtalk_message,
    "question": "발표에 대한 정보를 정리하세요.",
})

In [47]:
import json
print(json.dumps(answer.model_dump(), indent=4, ensure_ascii=False))

{
    "speaker": "이정호",
    "subject": "AI 패러다임 변화와 우리의 미래",
    "date": "2025-03-07 14:30:00"
}
